In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [ ]:
# Downloaded: https://towardsdatascience.com/downloading-openimages-dataset-in-google-drive-using-colab-132502f0dbde

In [ ]:
cd drive

/content/drive


In [ ]:
cd My\ Drive

/content/drive/My Drive


In [ ]:
!mkdir OpenImages


In [ ]:
cd OpenImages


/content/drive/My Drive/OpenImages


In [ ]:
!pwd


/content/drive/My Drive/OpenImages


In [ ]:
# Download required meta-files
!wget https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv
 
!wget https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv
 
!wget https://storage.googleapis.com/openimages/2018_04/validation/validation-annotations-bbox.csv
 
!wget https://storage.googleapis.com/openimages/2018_04/test/test-annotations-bbox.csv

--2020-11-09 10:37:55--  https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 172.217.218.128, 173.194.79.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11255 (11K) [text/csv]
Saving to: ‘class-descriptions-boxable.csv.3’

class-descriptions- 100%[===================>]  10.99K  --.-KB/s    in 0.002s  

2020-11-09 10:37:55 (6.41 MB/s) - ‘class-descriptions-boxable.csv.3’ saved [11255/11255]

--2020-11-09 10:37:55--  https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.69.128, 108.177.119.128, 108.177.126.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1194033454 (

In [ ]:
#This magic function is used to create the files.downloadOI.py is the file name
%%writefile downloadOI.py
#Author : Sunita Nayak, Big Vision LLC

#### Usage example: python3 downloadOI.py --classes 'Ice_cream,Cookie' --mode train

import argparse
import csv
import subprocess
import os
from tqdm import tqdm
import multiprocessing
from multiprocessing import Pool as thread_pool

cpu_count = multiprocessing.cpu_count()

parser = argparse.ArgumentParser(description='Download Class specific images from OpenImagesV4')
parser.add_argument("--mode", help="Dataset category - train, validation or test", required=True)
parser.add_argument("--classes", help="Names of object classes to be downloaded", required=True)
parser.add_argument("--nthreads", help="Number of threads to use", required=False, type=int, default=cpu_count*2)
parser.add_argument("--occluded", help="Include occluded images", required=False, type=int, default=1)
parser.add_argument("--truncated", help="Include truncated images", required=False, type=int, default=1)
parser.add_argument("--groupOf", help="Include groupOf images", required=False, type=int, default=1)
parser.add_argument("--depiction", help="Include depiction images", required=False, type=int, default=1)
parser.add_argument("--inside", help="Include inside images", required=False, type=int, default=1)

args = parser.parse_args()

run_mode = args.mode

threads = args.nthreads

classes = []
for class_name in args.classes.split(','):
    classes.append(class_name)

with open('./class-descriptions-boxable.csv', mode='r') as infile:
    reader = csv.reader(infile)
    dict_list = {rows[1]:rows[0] for rows in reader}

subprocess.run(['rm', '-rf', run_mode])
subprocess.run([ 'mkdir', run_mode])

pool = thread_pool(threads)
commands = []
cnt = 0

for ind in range(0, len(classes)):
    
    class_name = classes[ind]
    print("Class "+str(ind) + " : " + class_name)
    
    subprocess.run([ 'mkdir', run_mode+'/'+class_name])

    command = "grep "+dict_list[class_name.replace('_', ' ')] + " ./" + run_mode + "-annotations-bbox.csv"
    class_annotations = subprocess.run(command.split(), stdout=subprocess.PIPE).stdout.decode('utf-8')
    class_annotations = class_annotations.splitlines()

    for line in class_annotations:

        line_parts = line.split(',')
        
        #IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
        if (args.occluded==0 and int(line_parts[8])>0):
            print("Skipped %s",line_parts[0])
            continue
        if (args.truncated==0 and int(line_parts[9])>0):
            print("Skipped %s",line_parts[0])
            continue
        if (args.groupOf==0 and int(line_parts[10])>0):
            print("Skipped %s",line_parts[0])
            continue
        if (args.depiction==0 and int(line_parts[11])>0):
            print("Skipped %s",line_parts[0])
            continue
        if (args.inside==0 and int(line_parts[12])>0):
            print("Skipped %s",line_parts[0])
            continue

        cnt = cnt + 1

        command = 'aws s3 --no-sign-request --only-show-errors cp s3://open-images-dataset/'+run_mode+'/'+line_parts[0]+'.jpg '+ run_mode+'/'+class_name+'/'+line_parts[0]+'.jpg'
        commands.append(command)
        
        with open('%s/%s/%s.txt'%(run_mode,class_name,line_parts[0]),'a') as f:
            f.write(','.join([class_name, line_parts[4], line_parts[5], line_parts[6], line_parts[7]])+'\n')

print("Annotation Count : "+str(cnt))
commands = list(set(commands))
print("Number of images to be downloaded : "+str(len(commands)))

list(tqdm(pool.imap(os.system, commands), total = len(commands) ))

pool.close()
pool.join()

Overwriting downloadOI.py


In [ ]:
!ls

class-descriptions-boxable.csv	  train
class-descriptions-boxable.csv.1  train-annotations-bbox.csv
class-descriptions-boxable.csv.2  train-annotations-bbox.csv.1
class-descriptions-boxable.csv.3  train-annotations-bbox.csv.2
downloadOI.py			  train-annotations-bbox.csv.3
test				  validation
test-annotations-bbox.csv	  validation-annotations-bbox.csv
test-annotations-bbox.csv.1	  validation-annotations-bbox.csv.1
test-annotations-bbox.csv.2	  validation-annotations-bbox.csv.2


In [ ]:
!pip install awscli

In [ ]:
# Download Sink and Toilet images for test 
!python3 downloadOI.py --classes "Sink,Toilet" --mode test
# Download Sink and Toilet images for train
!python3 downloadOI.py --classes "Sink,Toilet" --mode train
# Download Sink and Toilet images for validation
!python3 downloadOI.py --classes "Sink,Toilet" --mode validation

In [ ]:
# Download Drill images for test 
!python3 downloadOI.py --classes "Laptop" --mode test
# Download Drill images for train
!python3 downloadOI.py --classes "Laptop" --mode train
# Download Drill images for validation
!python3 downloadOI.py --classes "Laptop" --mode validation

Class 0 : Laptop
Annotation Count : 199
Number of images to be downloaded : 177
100% 177/177 [05:46<00:00,  1.96s/it]
Class 0 : Laptop
Annotation Count : 9327
Number of images to be downloaded : 5528
100% 5528/5528 [2:58:24<00:00,  1.94s/it]
Class 0 : Laptop
Annotation Count : 61
Number of images to be downloaded : 47
100% 47/47 [01:40<00:00,  2.13s/it]


So we have three classes: Sink, Toilet and Laptop. In the next notebook i'm going to prepare these pictures.